In [1]:
import time
import re
import pylab as pl
import numpy as np
import pylab as pl

from datetime import datetime, timedelta

import subprocess

import os
import pandas as pd

import concurrent.futures


from joblib import Parallel, delayed
import shutil


from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import subprocess

In [ ]:
os.system('cd ../../; git add .; git commit -m "Back to PBFT style"; git push -f origin HEAD:main')


In [4]:
    # os.system('gcloud compute instances delete --zone=europe-west3-c --quiet $(gcloud compute instances list --filter="name~\'resdb\'" --format="value(name)")')
    # os.system('gcloud compute instances delete --zone=asia-south1-c --quiet $(gcloud compute instances list --filter="name~\'resdb\'" --format="value(name)")')
    # os.system('gcloud compute instances delete --zone=us-west1-b --quiet $(gcloud compute instances list --filter="name~\'resdb\'" --format="value(name)")')   
    # os.system('gcloud compute instances delete --zone=us-east5-c --quiet $(gcloud compute instances list --filter="name~\'resdb\'" --format="value(name)")')
    # os.system('gcloud compute instances delete --zone=asia-northeast1-b --quiet $(gcloud compute instances list --filter="name~\'resdb\'" --format="value(name)")')




In [5]:
# result = subprocess.run(command, shell=True, capture_output=True, text=True)

# if result.returncode == 0:
#     print("Operation successful!")
# else:
#     print(f"Error: {result.stderr}")

In [6]:
# time.sleep(80)

In [8]:
# time.sleep(60)

In [9]:
outputs = []

In [31]:
# config_list = ['localn_12cls']
# config_list = ['cre_setup_2_scale_1_node_info']
# config_list = ['rvc']


# config_list = ['localn_12cls']

# config_list = ['4node', '8node','test48']
config_list = [ '4node']

# config_list = ['test', 'test2']

# config_list2 = [\
#                 'local_12cls',\
# #                'global_3cls',\
# #                'global_4cls',\
# #                'global_6cls',\
#                'global_12cls',\
# #                'global_8cls',\
# #                'global_2cls',\
# #                'local_3cls',\
# #                'local_4cls',\
# #                'local_6cls',\
# #                'local_8cls',\
# #                'local_2cls'\
#                ]



config_list = config_list
# config_list = ['cre_setup_3_scale_1_node_info']




In [32]:
config_list

['4node']

In [33]:
# for cfig in config_list:
cfig = config_list[0]



os.system('rm node_info.csv')
os.system('cp node_setup/'+cfig+'.csv node_info.csv')




run_opt = 'n' #input('Enter mode (r or fresh from scratch default)')

# time.sleep(120)

def str2datetime(s):
    parts = s.split('.')
    dt = datetime.strptime(parts[0], "%Y-%m-%d %H:%M:%S")
    return dt.replace(microsecond=int(parts[1]))


def remove_outliers(x, outlierConstant = 1.5):
    a = np.array(x)
    upper_quartile = np.percentile(a, 75)
    lower_quartile = np.percentile(a, 25)
    IQR = (upper_quartile - lower_quartile) * outlierConstant
    quartileSet = (lower_quartile - IQR, upper_quartile + IQR)
    resultList = []
    removedList = []
    for y in a.tolist():
        if y >= quartileSet[0] and y <= quartileSet[1]:
            resultList.append(y)
        else:
            removedList.append(y)
    return (resultList, removedList)
def getThroughput(experiment):

    result = subprocess.check_output('cat ../'+experiment+'/nohup_client_*', shell=True)


    result = result.decode("utf-8") 

    lines = result.split('\n')

    commit_pat = re.compile('([^[].*) \[hotstuff info\] ([0-9.]*)$')
#     commit_pat = re.compile('([^[].*) \[hotstuff info\] got*')

    Times = []
    lats = []

    for line in lines:
        m = commit_pat.match(line)
        if m:
            try:

                time_item = float((line.split(' ')[1]).split(':')[0])*3600 + float((line.split(' ')[1]).split(':')[1])*60 + \
                float((line.split(' ')[1]).split(':')[2])
                Times.append(time_item)        

                lats.append(float(m.group(2)))
            except:
                continue


    begin_time = None
    values = []
    interval = 1
    cnt = 0
    Times.sort()
    for timestamp in Times:
        if begin_time is None:
            begin_time = timestamp
            next_begin_time = timestamp +interval

        while timestamp >= next_begin_time:
            begin_time = next_begin_time
            next_begin_time += interval
            values.append(cnt)
            cnt = 0
        cnt += 1
    values.append(cnt)

#     lats, _ = remove_outliers(lats)
#     print(len(values))


    return np.arange(len(values))*interval,np.array(values)/interval, lats

# p = subprocess.Popen(['../../hotstuff-keygen', ' --num', '--n', str(4)],
#                      stdout=subprocess.PIPE, stderr=open(os.devnull, 'w'))

# print([[t[4:] for t in l.decode('ascii').split()] for l in p.stdout])
# # [[t for t in l.decode('ascii').split()] for l in p.stdout.read()]

# os.system('echo 3108 | sudo -S sh job_mini.sh '+ 'test')


# regions= ['us-west-1', 'asia-south1-c']
# regions= ['asia-south1-c', 'eu-central-1', 'us-west-2']
# regions= ['asia-south1-c', 'eu-central-1']
regions= ['us-west1-b']


instance_type = 't2.small'

df =pd.read_csv('node_info.csv')

node_info_dict = df.to_dict()

print(node_info_dict)



# with open("cluster_info_hs.txt", 'w') as file:

#     for key in node_info_dict['cluster_id']:
#         print(key, node_info_dict['cluster_id'][key])

#         if node_info_dict['cluster_id'][key] >= 0:
#             file.write(str(node_info_dict['cluster_id'][key]) + '\n')



lines  = []



for idx, key in enumerate( node_info_dict['region'].keys()):
    cmd =  'gcloud compute instances create resdb-'+f"{idx:03}"+' --project=ucr-ursa-major-lesani-lab --zone='+node_info_dict['region'][key].strip()[1:-1]+' --machine-type=e2-standard-2 --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=goog-ec-src=vm_add-gcloud --reservation-affinity=any'
#     cmd = 'gcloud compute instances create resdb-'+f"{lim:02}"+' --project=ucr-ursa-major-lesani-lab --zone='+reg+' --machine-type=e2-small --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=goog-ec-src=vm_add-gcloud --reservation-affinity=any'

    lines.append(cmd)


lines


def run_command(command):
    subprocess.call(command, shell=True)

commands = lines

# # Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
# with concurrent.futures.ThreadPoolExecutor(max_workers=60) as executor:
#     # Submit each command to the executor for parallel execution
#     futures = [executor.submit(run_command, command) for command in commands]

#     # Wait for all tasks to complete
#     concurrent.futures.wait(futures)

# time.sleep(60)

instance_type = 'e2-small'


redirect = '>'

os.system('gcloud compute instances list --filter="name~\'resdb\'"   --format="value(networkInterfaces[0].networkIP)" '+\
          redirect+' all_internal_ips')



os.system('gcloud compute instances list --filter="name~\'resdb\'" --format="value(networkInterfaces[0].networkIP)" '+\
      redirect+' all_external_ips')







!cat all_internal_ips
! echo " " 
# !cat all_external_ips



f = open('all_external_ips', "r")
data1 = [line.strip() for line in f.readlines()]

data1 = [x for x in data1 if x!='None']

f = open('all_internal_ips', "r")
data2 = [line.strip() for line in f.readlines()]

data2 = [x for x in data2 if x!='None']



data2, len(data2)

data = []

# for i in range(len(data1)):
#     if data1[i]!='None':
# #         data.append(data1[i] + '    ' + data2[i])
#         data.append('127.0.0.1'+ '    ' + '127.0.0.1')


skip_instances = 0

for i in range(skip_instances, len(data1)):
    if data1[i]!='None':
        data.append(data1[i] + '    ' + data2[i])
#         data.append('127.0.0.1'+ '    ' + '127.0.0.1')

data, len(data)

for index, item in enumerate(data1):
    if 'publicIP' not in node_info_dict.keys():
        node_info_dict['publicIP'] = {}


    node_info_dict['publicIP'][index] = item 


for index, item in enumerate(data2):
    if 'privateIP' not in node_info_dict.keys():
        node_info_dict['privateIP'] = {}

    node_info_dict['privateIP'][index] = item 



for index, item in enumerate(data):
    if 'publicAndprivateIP' not in node_info_dict.keys():
        node_info_dict['publicAndprivateIP'] = {}

    node_info_dict['publicAndprivateIP'][index] = item 

node_info_dict

for item in node_info_dict['publicAndprivateIP']:
    print(node_info_dict['publicAndprivateIP'][item])

with open('replicas.txt', 'w') as f:

    for item in node_info_dict['publicAndprivateIP']:

        if (node_info_dict['ServerOrClientregion'][item] ==0):
            f.write("%s\n" % node_info_dict['publicAndprivateIP'][item])


node_info_dict['ServerOrClientregion']

! cat replicas.txt

with open('clients.txt', 'w') as f:

    for item in node_info_dict['publicIP']:

        if (node_info_dict['ServerOrClientregion'][item] ==1):
            f.write("%s\n" % node_info_dict['publicIP'][item])

os.system('cat clients.txt')

node_info_dict

def setup_remote(i):

    ip = node_info_dict['publicIP'][i] 
    if ip!='None':
        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo apt -y update;sudo apt -y install build-essential; sudo apt -y install autoconf libtool libssl-dev libuv1-dev cmake pkg-config cmake-data make; sudo rm -r hs; git clone https://github.com/tejas-shivanand-mane/O2PRBC.git;"'

        print(command)

        output = os.system(command)
        print(output)


def setup_remote2(i):

    ip = node_info_dict['publicIP'][i] 
    if ip!='None':
        current = (ip.split('.'))



        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; git pull;"'

        print(command)

        output = os.system(command)
        print(output)


def git_pull_remote(i):

    ip = node_info_dict['publicIP'][i] 
    if ip!='None':
        current = (ip.split('.'))
        command = 'ssh -o StrictHostKeyChecking=no -i /home/tejas/Documents/'+node_info_dict['key_file'][i].strip()[1:-1]+' '+'ubuntu@ec2-'+str(current[0])+'-'\
                      +str(current[1])+'-'+str(current[2])+'-'+str(current[3])+'.'+node_info_dict['region'][i].strip()[1:-1]+'.compute.amazonaws.com -T "cd O2PRBC; sudo git pull;"'


        output = subprocess.getoutput(command)
        print(output)


def kill_nodes_remote(i):
    ip = node_info_dict['publicIP'][i] 

    if ip!='None':
        current = (ip.split('.'))

        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo killall hotstuff-app; sudo killall hotstuff-client;"'


        output = subprocess.getoutput(command)
        print(output)



def clean_nodes_remote(i):
    ip = node_info_dict['publicIP'][i] 

    if ip!='None':
        current = (ip.split('.'))

        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo rm -f nohup.out;"'


        print(command)
        os.system(command)

<>:45: SyntaxWarning: invalid escape sequence '\['
<>:45: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_96757/1784368491.py:45: SyntaxWarning: invalid escape sequence '\['
  commit_pat = re.compile('([^[].*) \[hotstuff info\] ([0-9.]*)$')


{'node_id': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, 'region': {0: " 'europe-west3-c'", 1: " 'europe-west3-c'", 2: " 'europe-west3-c'", 3: " 'europe-west3-c'", 4: " 'europe-west3-c'", 5: " 'europe-west3-c'", 6: " 'europe-west3-c'", 7: " 'europe-west3-c'"}, 'ServerOrClientregion': {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1}, 'cluster_id': {0: 0, 1: 0, 2: 0, 3: 0, 4: -1, 5: -1, 6: -1, 7: -1}, 'key_file': {0: " 'FKA.pem'", 1: " 'FKA.pem'", 2: " 'FKA.pem'", 3: " 'FKA.pem'", 4: " 'FKA.pem'", 5: " 'FKA.pem'", 6: " 'FKA.pem'", 7: " 'FKA.pem'"}, 'clientID': {0: -1, 1: -1, 2: -1, 3: -1, 4: 0, 5: 1, 6: 2, 7: 3}, 'ServerID': {0: 0, 1: 1, 2: 2, 3: 3, 4: -1, 5: -1, 6: -1, 7: -1}, ' PublicIp': {0: nan, 1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: nan, 7: nan}}
10.156.0.23
10.156.0.37
10.156.0.66
10.156.0.41
10.156.0.22
10.156.0.55
10.156.0.65
10.156.0.46
 
10.156.0.23    10.156.0.23
10.156.0.37    10.156.0.37
10.156.0.66    10.156.0.66
10.156.0.41    10.156.0.41
10.156.0.22    10.156.0.2

In [34]:
experiment = 'test'
os.system('rm hotstuff.gen*')
os.system('echo 3108 | sudo -S sh job_mini.sh '+ experiment)


os.system('cp hotstuff.gen* ../../')


os.system('cd ../../; git add .; git commit -m "test"; git push -f origin HEAD:main')

prefix = 'O2PRBC_normal_converted_'
    

Name of TestRun: test
-- Found OpenSSL: /usr/lib/x86_64-linux-gnu/libcrypto.so (found suitable version "3.4.1", minimum required is "1.1.0")
-- Found OpenSSL: /usr/lib/x86_64-linux-gnu/libcrypto.so (found version "3.4.1")
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Configuring done (0.1s)
-- Generating done (0.0s)
-- Build files have been written to: /home/tejas/work/O2PRBC
[ 21%] Built target libsecp256k1


[sudo] password for tejas: CMake Warning:
  No source or binary directory provided.  Both will be assumed to be the
  same as the current working directory, but note that this warning will
  become a fatal error in future CMake releases.


CMake Deprecation Warning at CMakeLists.txt:16 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


CMake Deprecation Warning at salticidae/CMakeLists.txt:23 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.




[ 37%] Built target hotstuff
[ 62%] Built target salticidae
[ 67%] Built target salticidae_shared
[ 67%] Built target salticidae_static
[ 72%] Built target hotstuff_static
[ 72%] Built target hotstuff_shared
[ 83%] Built target hotstuff-tls-keygen
[ 83%] Built target test_secp256k1
[ 89%] Built target hotstuff-keygen
[ 94%] Built target hotstuff-app
[100%] Built target hotstuff-client
[main 87b81bd] test
 5 files changed, 16 insertions(+), 20 deletions(-)


To github.com:tejas-shivanand-mane/O2PRBC.git
   0d7607a..87b81bd  HEAD -> main


In [35]:
run_opt ='r'

results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
                                                                       for i in node_info_dict['publicIP'])


# for i in node_info_dict['publicIP']:
#     print(i)
#     kill_nodes_remote(i)


if run_opt == 'r':
    results = Parallel(n_jobs=60)(delayed(setup_remote2)(i)\
                                                                       for i in node_info_dict['node_id'])

else:
    results = Parallel(n_jobs=60)(delayed(setup_remote)(i)\
                                                                       for i in node_info_dict['node_id'])


print(results)  

# for i in node_info_dict['publicIP']:
#     print(i)
#     setup_remote2(i)

# time.process_time()



def compile_remote(i):

    ip = node_info_dict['publicIP'][i] 
    if ip!='None':
        current = (ip.split('.'))


        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup sh compile_job.sh >../make_job.out 2>&1 &"'


        print(command)
        output = os.system(command)
        print(output)







results = Parallel(n_jobs=60)(delayed(compile_remote)(i)\
                                                                       for i in list(node_info_dict['node_id'].keys()))

# for i in  list(node_info_dict['node_id'].keys()):
#     compile_remote(i)

print(results)  


t_compile =  0

if run_opt=='r':
    t_compile = 60
else:
    t_compile = 150

t_compile

time.sleep(t_compile)

results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
                                                                       for i in node_info_dict['publicIP'])



results = Parallel(n_jobs=60 )(delayed(clean_nodes_remote)(i)\
                                                                       for i in node_info_dict['node_id'])


print(results)  


# node_info_dict

def run_server_node_remote(i):
    ip = node_info_dict['publicIP'][i] 
#     if ip!='None' and( ip in nodeIps[:-3]):

    if ip!='None' and node_info_dict['ServerOrClientregion'][i]==0:

        current = (ip.split('.'))


        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup ./examples/hotstuff-app --conf hotstuff.gen-sec'+str(node_info_dict['ServerID'][i])+'.conf >../nohup.out 2>&1 &"'

        print(i,command)
        os.system(command)


results = Parallel(n_jobs=60)(delayed(run_server_node_remote)(i)\
for i in node_info_dict['node_id'])

print(results)  
time.sleep(45)

def run_client_node_remote(i):
    ip = node_info_dict['publicIP'][i] 

    if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:

        current = (ip.split('.'))


        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup ./examples/hotstuff-client --idx '+str(node_info_dict['clientID'][i])+' --iter -1 --max-async 400 >>../nohup.out 2>&1 &"'





        print(command)
        os.system(command)



#         results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
#                            for i in [17])



results = Parallel(n_jobs=60)(delayed(run_client_node_remote)(i)\
               for i in node_info_dict['node_id'])
time.sleep(80)
#         results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
#                                    for i in [18,19])


print(results)  



# results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
#                                                                        for i in [4])


time.sleep(20)



# def run_client_node_remote(i):
#     ip = node_info_dict['publicIP'][i] 

#     if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:

#         current = (ip.split('.'))


#         command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; sudo nohup ./examples/hotstuff-client --idx '+str(node_info_dict['clientID'][i])+' --iter -1 --max-async 100 >>../nohup1.out 2>&1 &"'





#         print(command)
#         os.system(command)



# #         results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
# #                            for i in [17])



# results = Parallel(n_jobs=60)(delayed(run_client_node_remote)(i)\
#                for i in node_info_dict['node_id'])
# time.sleep(100)



results = Parallel(n_jobs=60)(delayed(kill_nodes_remote)(i)\
                                                                       for i in node_info_dict['publicIP'])




# for i in node_info_dict['publicIP']:
#     print(i)
#     kill_nodes_remote(i)

os.system('cd ../../../dump_files; echo 3108 | sudo -S  rm *')

#         for i in node_info_dict['publicIP']:
#             ip = node_info_dict['publicIP'][i] 

#             if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
#                 current = (ip.split('.'))
#                 command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup.out ../../../dump_files/nohup_client_'+str(node_info_dict['clientID'][i])+'.out'
#                 print(command)
#                 os.system(command)

#         break






experiment = cfig

if not os.path.exists('../../../Experiments/'+prefix+experiment):
    os.mkdir('../../../Experiments/'+prefix+experiment)








temp_experiment = '../../dump_files'






os.system('cp -r ../../../dump_files ../../../Experiments/'+prefix+experiment)    



#         os.system('gcloud compute instances delete --zone=europe-west3-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=asia-south1-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=us-west1-b --quiet $(gcloud compute instances list --format="value(name)")')   

#     except:


#         experiment = cfig

#         if not os.path.exists('../../../Experiments/h_ours'+experiment):
#             os.mkdir('../../../Experiments/h_ours'+experiment)

#         os.system('gcloud compute instances delete --zone=europe-west3-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=asia-south1-c --quiet $(gcloud compute instances list --format="value(name)")')
#         os.system('gcloud compute instances delete --zone=us-west1-b --quiet $(gcloud compute instances list --format="value(name)")')

#         continue


for i in node_info_dict['publicIP']:
    ip = node_info_dict['publicIP'][i] 

    if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
        current = (ip.split('.'))
        command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup.out ../../../dump_files/nohup_client_'+str(node_info_dict['clientID'][i])+'.out'
        os.system(command)
        
        
        command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup1.out ../../../dump_files/nohup_client_'+str(1+node_info_dict['clientID'][i])+'.out'
        os.system(command)


os.system('cp -r ../../../dump_files ../../../Experiments/'+prefix+experiment)    


for i in node_info_dict['publicIP']:
    ip = node_info_dict['publicIP'][i] 

    if ip!='None' and node_info_dict['ServerOrClientregion'][i]==0:
        current = (ip.split('.'))
        command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup.out ../../../dump_files/nohup_server_'+str(node_info_dict['ServerID'][i])+'.out'
#         substrings = ['_1.', '_5.', '_9.', '_13.', '_17.', '_21.']
#         if any(substring in command for substring in substrings):

#             print(command)

        os.system(command)
        break


os.system('cp -r ../../../dump_files ../../../Experiments/'+prefix+experiment)  

From https://github.com/tejas-shivanand-mane/O2PRBC
   0d7607a..87b81bd  main       -> origin/main


Updating 0d7607a..87b81bd
Fast-forward
 hotstuff.gen-sec0.conf |  6 +++---
 hotstuff.gen-sec1.conf |  6 +++---
 hotstuff.gen-sec2.conf |  6 +++---
 hotstuff.gen-sec3.conf |  6 +++---
 hotstuff.gen.conf      | 12 ++++--------
 5 files changed, 16 insertions(+), 20 deletions(-)


From https://github.com/tejas-shivanand-mane/O2PRBC
   0d7607a..87b81bd  main       -> origin/main
From https://github.com/tejas-shivanand-mane/O2PRBC
   0d7607a..87b81bd  main       -> origin/main
From https://github.com/tejas-shivanand-mane/O2PRBC
   0d7607a..87b81bd  main       -> origin/main
From https://github.com/tejas-shivanand-mane/O2PRBC
   0d7607a..87b81bd  main       -> origin/main


Updating 0d7607a..87b81bd
Fast-forward
 hotstuff.gen-sec0.conf |  6 +++---
 hotstuff.gen-sec1.conf |  6 +++---
 hotstuff.gen-sec2.conf |  6 +++---
 hotstuff.gen-sec3.conf |  6 +++---
 hotstuff.gen.conf      | 12 ++++--------
 5 files changed, 16 insertions(+), 20 deletions(-)
Updating 0d7607a..87b81bd
Fast-forward
 hotstuff.gen-sec0.conf |  6 +++---
 hotstuff.gen-sec1.conf |  6 +++---
 hotstuff.gen-sec2.conf |  6 +++---
 hotstuff.gen-sec3.conf |  6 +++---
 hotstuff.gen.conf      | 12 ++++--------
 5 files changed, 16 insertions(+), 20 deletions(-)
Updating 0d7607a..87b81bd
Fast-forward
 hotstuff.gen-sec0.conf |  6 +++---
 hotstuff.gen-sec1.conf |  6 +++---
 hotstuff.gen-sec2.conf |  6 +++---
 hotstuff.gen-sec3.conf |  6 +++---
 hotstuff.gen.conf      | 12 ++++--------
 5 files changed, 16 insertions(+), 20 deletions(-)
Updating 0d7607a..87b81bd
Fast-forward
 hotstuff.gen-sec0.conf |  6 +++---
 hotstuff.gen-sec1.conf |  6 +++---
 hotstuff.gen-sec2.conf |  6 +++---
 hotstuff.gen-sec3.con

From https://github.com/tejas-shivanand-mane/O2PRBC
   0d7607a..87b81bd  main       -> origin/main
From https://github.com/tejas-shivanand-mane/O2PRBC
   0d7607a..87b81bd  main       -> origin/main
From https://github.com/tejas-shivanand-mane/O2PRBC
   0d7607a..87b81bd  main       -> origin/main


Updating 0d7607a..87b81bd
Fast-forward
 hotstuff.gen-sec0.conf |  6 +++---
 hotstuff.gen-sec1.conf |  6 +++---
 hotstuff.gen-sec2.conf |  6 +++---
 hotstuff.gen-sec3.conf |  6 +++---
 hotstuff.gen.conf      | 12 ++++--------
 5 files changed, 16 insertions(+), 20 deletions(-)
Updating 0d7607a..87b81bd
Fast-forward
 hotstuff.gen-sec0.conf |  6 +++---
 hotstuff.gen-sec1.conf |  6 +++---
 hotstuff.gen-sec2.conf |  6 +++---
 hotstuff.gen-sec3.conf |  6 +++---
 hotstuff.gen.conf      | 12 ++++--------
 5 files changed, 16 insertions(+), 20 deletions(-)
Updating 0d7607a..87b81bd
Fast-forward
 hotstuff.gen-sec0.conf |  6 +++---
 hotstuff.gen-sec1.conf |  6 +++---
 hotstuff.gen-sec2.conf |  6 +++---
 hotstuff.gen-sec3.conf |  6 +++---
 hotstuff.gen.conf      | 12 ++++--------
 5 files changed, 16 insertions(+), 20 deletions(-)
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, Non

[sudo] password for tejas: /usr/bin/scp: /home/tejas/nohup1.out: No such file or directory
ERROR: (gcloud.compute.scp) [/usr/bin/scp] exited with return code [1].
/usr/bin/scp: /home/tejas/nohup1.out: No such file or directory
ERROR: (gcloud.compute.scp) [/usr/bin/scp] exited with return code [1].


hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-004" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; git pull;"
0
gcloud compute ssh --zone "europe-west3-c" "resdb-003" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; git pull;"
0
gcloud compute ssh --zone "europe-west3-c" "resdb-001" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; git pull;"
0
gcloud compute ssh --zone "europe-west3-c" "resdb-002" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; git pull;"
0
gcloud compute ssh --zone "europe-west3-c" "resdb-007" --project "ucr-ursa-major-lesani-lab" --command "cd O2PRBC; git pull;"
0
gcloud compute ssh --zone "europe-west3-c" "resdb-005" --project "ucr-ursa-major-lesani-lab" --comma

/usr/bin/scp: /home/tejas/nohup1.out: No such file or directory
ERROR: (gcloud.compute.scp) [/usr/bin/scp] exited with return code [1].
/usr/bin/scp: /home/tejas/nohup1.out: No such file or directory
ERROR: (gcloud.compute.scp) [/usr/bin/scp] exited with return code [1].


0

In [36]:
'../../../Experiments/'+prefix+experiment

'../../../Experiments/O2PRBC_normal_converted_4node'

In [37]:
    # for i in node_info_dict['publicIP']:
    #     ip = node_info_dict['publicIP'][i] 
    
    #     if ip!='None' and node_info_dict['ServerOrClientregion'][i]==1:
    #         current = (ip.split('.'))
    #         command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup.out ../../../dump_files/nohup_client_'+str(node_info_dict['clientID'][i])+'.out'
    #         os.system(command)
            
            
    #         command = 'gcloud compute scp --zone "'+str(node_info_dict['region'][i].strip()[1:-1]) +'"'+' resdb-'+f"{i:03}"+':/home/tejas/nohup1.out ../../../dump_files/nohup_client_'+str(1+node_info_dict['clientID'][i])+'.out'
    #         os.system(command)
    
    
    # os.system('cp -r ../../../dump_files ../../../Experiments/'+prefix+experiment)    

hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
hotstuff-app: no process found
hotstuff-client: no process found
gcloud compute ssh --zone "europe-west3-c" "resdb-005" --project "ucr-ursa-major-lesani-lab" --command "sudo rm -f nohup.out;"
gcloud compute ssh --zone "europe-west3-c" "resdb-002" --project "ucr-ursa-major-lesani-lab" --command "sudo rm -f nohup.out;"
gcloud compute ssh --zone "europe-west3-c" "resdb-003" --project "ucr-ursa-major-lesani-lab" --command "sudo rm -f nohup.out;"
gcloud compute ssh --zone "europe-west3-c" "resdb-001" --project "ucr-ursa-major-lesani-lab" --comm